In [19]:
%pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# ===== PARAMÈTRES DE CONFIGURATION =====
# Ajustez ces valeurs selon vos besoins

# Nombre d'entités à générer
NOMBRE_UTILISATEURS = 80  # Nombre d'utilisateurs à créer
NOMBRE_ACTIVITES = 50     # Nombre d'activités à créer
NOMBRE_EVENEMENTS = 5     # Nombre d'événements à créer (en quantité)

# Périodes pour les sessions
SESSIONS_JOURS_PASSE_MIN = 1   # Sessions dans le passé (minimum X jours avant)
SESSIONS_JOURS_PASSE_MAX = 365  # Sessions dans le passé (maximum X jours avant)
SESSIONS_JOURS_FUTUR_MIN = 1    # Sessions programmées dans au minimum X jours
SESSIONS_JOURS_FUTUR_MAX = 60   # Sessions programmées dans au maximum X jours
SESSIONS_PAR_ACTIVITE_MIN = 1   # Minimum de sessions par activité
SESSIONS_PAR_ACTIVITE_MAX = 12   # Maximum de sessions par activité
POURCENTAGE_SESSIONS_PASSEES = 65  # Pourcentage de sessions dans le passé

# Jours de la semaine autorisés pour les sessions (0=Lundi, 1=Mardi, ..., 6=Dimanche)
# Modifiez cette liste pour activer/désactiver certains jours
JOURS_AUTORISES = [4, 5, 6]  # Tous les jours par défaut
# Exemples d'autres configurations :
# JOURS_AUTORISES = [2, 4, 5]  # Seulement mercredi, vendredi, samedi
# JOURS_AUTORISES = [5, 6]     # Seulement weekend (samedi, dimanche)
# JOURS_AUTORISES = [0, 1, 2, 3, 4]  # Seulement en semaine (lundi à vendredi)

# Heures des sessions (0-23)
HEURE_DEBUT_MIN = 14  # Heure de début la plus tôt (14h)
HEURE_DEBUT_MAX = 19  # Heure de début la plus tard (19h)
HEURE_FIN_MIN = 0     # Heure de fin la plus tôt (0h = minuit)
HEURE_FIN_MAX = 2    # Heure de fin la plus tard (23h)

# Types d'activités
NOMBRE_JEUX_DE_SOCIETE = 30      # Nombre d'activités "JEU_DE_SOCIETE" à créer
POURCENTAGE_CAMPAGNES_FERMEES = 65  # Pourcentage de campagnes fermées (sur 100)

# Autres paramètres
POURCENTAGE_ADMIN = 10           # Pourcentage d'administrateurs (sur 100)
TAUX_INSCRIPTION_SESSIONS = 0.9  # Taux de remplissage moyen des sessions (0.0 à 1.0)

# Fonction utilitaire pour afficher les jours de la semaine
def afficher_jours_autorises(jours_list):
    noms_jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    return ', '.join([noms_jours[jour] for jour in sorted(jours_list)])

print(f"📊 Configuration de génération:")
print(f"   • {NOMBRE_UTILISATEURS} utilisateurs")
print(f"   • {NOMBRE_ACTIVITES} activités (dont {NOMBRE_JEUX_DE_SOCIETE} jeux de société)")
print(f"   • {NOMBRE_EVENEMENTS} événements")
print(f"   • Sessions passées: {SESSIONS_JOURS_PASSE_MIN}-{SESSIONS_JOURS_PASSE_MAX} jours ({POURCENTAGE_SESSIONS_PASSEES}%)")
print(f"   • Sessions futures: {SESSIONS_JOURS_FUTUR_MIN}-{SESSIONS_JOURS_FUTUR_MAX} jours")
print(f"   • {SESSIONS_PAR_ACTIVITE_MIN} à {SESSIONS_PAR_ACTIVITE_MAX} sessions par activité")
print(f"   • Jours autorisés: {afficher_jours_autorises(JOURS_AUTORISES)}")
print(f"   • Heures: {HEURE_DEBUT_MIN}h-{HEURE_DEBUT_MAX}h → {HEURE_FIN_MIN}h-{HEURE_FIN_MAX}h")
print(f"   • {POURCENTAGE_ADMIN}% d'administrateurs")
print(f"   • Taux de remplissage sessions: {int(TAUX_INSCRIPTION_SESSIONS*100)}%")

📊 Configuration de génération:
   • 80 utilisateurs
   • 50 activités (dont 30 jeux de société)
   • 5 événements
   • Sessions passées: 1-365 jours (65%)
   • Sessions futures: 1-60 jours
   • 1 à 12 sessions par activité
   • Jours autorisés: Vendredi, Samedi, Dimanche
   • Heures: 14h-19h → 0h-2h
   • 10% d'administrateurs
   • Taux de remplissage sessions: 90%


In [17]:
# Script de génération de données de test pour la base de données LBDR
import mysql.connector
import random
import uuid
from datetime import datetime, timedelta, date
import json

# Configuration de la connexion à la base de données
config = {
    'user': 'user',
    'password': 'userpassword',
    'host': 'localhost',
    'database': 'lbdr_db',
    'port': 3306
}

# Définition d'un dictionnaire pour associer les chemins d'images aux jeux correspondants
images_jeux = {
    "Warhammer": "https://cdn1.epicgames.com/offer/f640a0c1648147fea7e81565b45a3003/EGS_Warhammer40000SpaceMarine2_SaberInteractive_S1_2560x1440-975214651d1d1bc6c6e5779b53958840",
    "L'Appel de Cthulhu": "/data/images/creature-cthulhu-grand-ancien.webp",
    "Donjons et Dragons": "/data/images/donjon&dragon.jpg",
    "Vampire: La Mascarade": "/data/images/vampire.webp",
    "Chroniques Oubliées": "/data/images/Chroniques_Oubliees.jpg",
    "Shadowrun": "/data/images/Shadowrun.jpg",
    "Numenera": "/data/images/Numenera.jpg",
    "Blades in the Dark": "/data/images/Blades in the Dark.jpg",
    "Cyberpunk RED": "/data/images/Cyberpunk RED.png",
    "Pathfinder": "/data/images/Pathfinder.webp"
}

# Dictionnaire pour les icônes des jeux
icones_jeux = {
    "Warhammer": "/data/images/icon/Warhammer40k.png",
    "Donjons et Dragons": "/data/images/icon/dnd_logo_big.webp"
    # Les autres jeux n'ont pas d'icônes ou elles sont commentées dans le fichier games.tsx
}

try:
    # Connexion à la base de données
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    print("Connexion à la base de données réussie!")
    
        # Vider toutes les tables de la base de données dans l'ordre des dépendances
    tables_a_vider = [
        # D'abord les tables avec des clés étrangères
        "refresh_tokens",
        "paiements_helloasso",
        "notifications_helloasso",
        "creneaux_utilisateur",
        "horaires_lieu",
        "joueurs_session",
        "membres_activite",
        "sessions",
        "jeux_genres",
        "evenements",
        "activites",
        # Puis les tables principales
        "lieux",
        "genres",
        "jeux",
        "periodes_association",
        "utilisateurs"
    ]

    # Désactiver temporairement les contraintes de clés étrangères pour faciliter le nettoyage
    cursor.execute("SET FOREIGN_KEY_CHECKS=0")
    
    for table in tables_a_vider:
        try:
            cursor.execute(f"DELETE FROM {table}")
            print(f"✓ Table {table} vidée")
        except mysql.connector.Error as err:
            print(f"! Erreur lors du nettoyage de la table {table}: {err}")
    
    # Réactiver les contraintes de clés étrangères
    cursor.execute("SET FOREIGN_KEY_CHECKS=1")

    conn.commit()
    print("✅ Toutes les tables ont été vidées avec succès!")
    
    # --- FONCTION UTILITAIRES ---
    def generate_password_hash():
        """Génère un hash de mot de passe fictif"""
        return ''.join(random.choice('0123456789abcdef') for _ in range(64))
    
    def random_date(start_date, end_date):
        """Génère une date aléatoire entre start_date et end_date"""
        time_between_dates = end_date - start_date
        days_between_dates = time_between_dates.days
        random_number_of_days = random.randrange(days_between_dates)
        return start_date + timedelta(days=random_number_of_days)
    
    def generer_date_autorisee(is_passee=True, max_tentatives=100):
        """Génère une date qui respecte les jours autorisés"""
        for _ in range(max_tentatives):
            if is_passee:
                # Session dans le passé
                jours_passe = random.randint(SESSIONS_JOURS_PASSE_MIN, SESSIONS_JOURS_PASSE_MAX)
                date_candidate = datetime.now() - timedelta(days=jours_passe)
            else:
                # Session dans le futur
                jours_futur = random.randint(SESSIONS_JOURS_FUTUR_MIN, SESSIONS_JOURS_FUTUR_MAX)
                date_candidate = datetime.now() + timedelta(days=jours_futur)
            
            # Vérifier si le jour de la semaine est autorisé (0=Lundi, 6=Dimanche)
            jour_semaine = date_candidate.weekday()
            if jour_semaine in JOURS_AUTORISES:
                return date_candidate.strftime('%Y-%m-%d')
        
        # Si aucune date valide n'est trouvée après max_tentatives, 
        # on retourne une date avec le premier jour autorisé
        if JOURS_AUTORISES:
            jour_cible = JOURS_AUTORISES[0]
            if is_passee:
                jours_passe = random.randint(SESSIONS_JOURS_PASSE_MIN, SESSIONS_JOURS_PASSE_MAX)
                date_base = datetime.now() - timedelta(days=jours_passe)
            else:
                jours_futur = random.randint(SESSIONS_JOURS_FUTUR_MIN, SESSIONS_JOURS_FUTUR_MAX)
                date_base = datetime.now() + timedelta(days=jours_futur)
            
            # Ajuster la date pour qu'elle tombe sur un jour autorisé
            jours_a_ajouter = (jour_cible - date_base.weekday()) % 7
            date_ajustee = date_base + timedelta(days=jours_a_ajouter)
            return date_ajustee.strftime('%Y-%m-%d')
        
        # Fallback si aucun jour n'est autorisé (ne devrait pas arriver)
        return datetime.now().strftime('%Y-%m-%d')
    
    # --- GÉNÉRATION DES UTILISATEURS ---
    print("Génération des utilisateurs...")
    
    # Liste de prénoms et noms pour la génération
    prenoms = ['Alice', 'Bob', 'Charlie', 'David', 'Emma', 'François', 'Gilles', 'Hélène', 
               'Isabelle', 'Jean', 'Kevin', 'Lucie', 'Marc', 'Nathalie', 'Olivier', 'Patricia']
    noms = ['Martin', 'Dupont', 'Durand', 'Lefebvre', 'Moreau', 'Simon', 'Laurent', 
            'Michel', 'Leroy', 'Garcia', 'Bernard', 'Thomas', 'Robert', 'Richard']
    sexes = ['M', 'F', 'Autre']
    types_utilisateur = ['INSCRIT', 'INSCRIT', 'INSCRIT', 'ADMINISTRATEUR']  # Plus de chance d'avoir des inscrits

    # Génération d'utilisateurs
    users_data = []
    NOMBRE_UTILISATEURS = 20  # Paramètre pour le nombre d'utilisateurs à créer
    POURCENTAGE_ADMIN = 10  # Pourcentage d'utilisateurs qui seront des administrateurs
    
    for i in range(NOMBRE_UTILISATEURS):  # Utilisation du paramètre
        user_id = str(uuid.uuid4())
        prenom = random.choice(prenoms)
        nom = random.choice(noms)
        login = f"{prenom.lower()}.{nom.lower()}{random.randint(1, 999)}"
        date_naissance = random_date(date(1970, 1, 1), date(2005, 12, 31))
        sexe = random.choice(sexes)
        id_discord = f"{prenom.lower()}#{random.randint(1000, 9999)}" if random.random() > 0.3 else None
        pseudonyme = f"{prenom.lower()}{random.randint(1, 99)}" if random.random() > 0.3 else None
        email = f"{login}@example.com" if random.random() > 0.2 else None
        mot_de_passe = generate_password_hash()
        image = None  # Pour simplifier
        
        # Utilisation du paramètre pour déterminer le type d'utilisateur
        if random.randint(1, 100) <= POURCENTAGE_ADMIN:
            type_utilisateur = 'ADMINISTRATEUR'
        else:
            type_utilisateur = 'INSCRIT'
            
        date_inscription = random_date(date(2020, 1, 1), date(2025, 5, 1))
        ancien_utilisateur = random.random() > 0.7
        premiere_connexion = random.random() > 0.8
        
        users_data.append((
            user_id, prenom, nom, login, date_naissance, sexe, id_discord, pseudonyme,
            email, mot_de_passe, image, type_utilisateur, date_inscription,
            ancien_utilisateur, premiere_connexion
        ))
    
    # Insertion des utilisateurs dans la base de données
    user_sql = """
    INSERT INTO utilisateurs (
        id, prenom, nom, login, date_de_naissance, sexe, id_discord, pseudonyme,
        email, mot_de_passe, image, type_utilisateur, date_inscription,
        ancien_utilisateur, premiere_connexion
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(user_sql, users_data)
    conn.commit()
    print(f"✅ {len(users_data)} utilisateurs créés avec succès!")
    

    # --- GÉNÉRATION DES JEUX ---
    print("Génération des jeux...")
    
    jeux_base = [
        # Jeux de rôle
        ("Donjons et Dragons", "Le jeu de rôle fantasy le plus populaire au monde", "JDR"),
        ("L'Appel de Cthulhu", "Jeu d'horreur lovecraftienne", "JDR"),
        ("Warhammer Fantasy", "Un monde de dark fantasy médiévale", "JDR"),
        ("Pathfinder", "Un jeu de rôle d'heroic fantasy", "JDR"),
        ("Vampire: La Mascarade", "Un jeu gothique-punk où vous incarnez des vampires", "JDR"),
        ("Cyberpunk RED", "Un jeu de rôle futuriste dans un monde dystopique", "JDR"),
        ("Star Wars: Edge of the Empire", "Aventures dans l'univers de Star Wars", "JDR"),
        ("Chroniques Oubliées", "Un jeu de rôle médiéval-fantastique accessible", "JDR"),
        ("Shadowrun", "Mélange de cyberpunk et de fantasy avec magie et technologie", "JDR"),
        ("Blades in the Dark", "Un jeu de criminels dans une cité victorienne hantée", "JDR"),
        ("Tales from the Loop", "Aventures d'enfants dans les années 80 alternatives", "JDR"),
        ("Loup-Garou l'Apocalypse", "Incarnez des guerriers-loups défenseurs de la nature", "JDR"),
        ("Le Seigneur des Anneaux JdR", "Aventures dans la Terre du Milieu", "JDR"),
        ("Alien RPG", "Horreur et survie dans l'espace", "JDR"),
        ("Numenera", "Science-fantasy dans un futur très lointain", "JDR"),
        
        # Jeux de société
        ("Catan", "Jeu de société de stratégie et de développement", "JEU_DE_SOCIETE"),
        ("7 Wonders", "Jeu de cartes et de civilisations", "JEU_DE_SOCIETE"),
        ("Pandemic", "Jeu coopératif pour sauver l'humanité", "JEU_DE_SOCIETE"),
        ("Terraforming Mars", "Jeu de stratégie sur la colonisation de Mars", "JEU_DE_SOCIETE"),
        ("Carcassonne", "Jeu de tuiles et de territoire", "JEU_DE_SOCIETE"),
        ("Dixit", "Jeu d'imagination et d'interprétation d'images", "JEU_DE_SOCIETE"),
        ("Splendor", "Jeu de développement économique dans la Renaissance", "JEU_DE_SOCIETE"),
        ("Les Aventuriers du Rail", "Construction de routes ferroviaires", "JEU_DE_SOCIETE"),
        ("Azul", "Jeu de placement de tuiles aux motifs inspirés d'azulejos", "JEU_DE_SOCIETE"),
        ("Codenames", "Jeu d'association de mots en équipes", "JEU_DE_SOCIETE"),
        ("King of Tokyo", "Affrontement de monstres géants", "JEU_DE_SOCIETE"),
        ("Dominion", "Jeu de construction de deck médiéval", "JEU_DE_SOCIETE"),
        ("Scythe", "Conquête de territoires dans un monde uchronique", "JEU_DE_SOCIETE"),
        ("Root", "Lutte asymétrique pour le contrôle de la forêt", "JEU_DE_SOCIETE"),
        ("Wingspan", "Collection d'oiseaux et création d'habitat", "JEU_DE_SOCIETE")
    ]
    
    # Préparation des données de jeux avec les images ET icônes
    jeux_data = []
    for nom_jeu, description, type_jeu in jeux_base:
        # Chercher si le jeu a une image associée dans notre dictionnaire
        image = None
        icon = None
        
        # Gérer le cas de Warhammer (Warhammer Fantasy dans la base, mais Warhammer dans le dictionnaire d'images)
        recherche = "Warhammer" if nom_jeu.startswith("Warhammer") else nom_jeu
        
        # Pour Vampire, le nom exact est "Vampire: La Mascarade"
        if recherche == "Vampire: La Mascarade":
            image = images_jeux.get(recherche)
            icon = icones_jeux.get(recherche)
        else:
            # Recherche activitelle pour les autres jeux
            for cle in images_jeux:
                if cle in recherche:
                    image = images_jeux[cle]
                    break
            for cle in icones_jeux:
                if cle in recherche:
                    icon = icones_jeux[cle]
                    break
        
        jeux_data.append((nom_jeu, description, type_jeu, image, icon))
    
    jeux_sql = """
    INSERT INTO jeux (nom, description, type_jeu, image, icon)
    VALUES (%s, %s, %s, %s, %s)
    """
    
    cursor.executemany(jeux_sql, jeux_data)
    conn.commit()
    print(f"✅ {len(jeux_data)} jeux créés avec succès!")
    
    # --- OBTENTION DES IDS DES JEUX CRÉÉS ---
    cursor.execute("SELECT id, nom, image FROM jeux")
    jeux_info = cursor.fetchall()
    jeux_ids = [row[0] for row in jeux_info]
    
    # Créer un dictionnaire pour lier les ID des jeux à leurs images
    jeux_images_dict = {row[0]: row[2] for row in jeux_info}
    
        # --- GÉNÉRATION DES GENRES ---
    print("Génération des genres...")
    
    # Définition des genres pour les jeux de rôle et de société
    genres_list = [
        "Fantastique", 
        "Horreur", 
        "Science-fiction", 
        "Post-apocalyptique",
        "Médiéval", 
        "Contemporain", 
        "Steampunk", 
        "Cyberpunk", 
        "Heroic-Fantasy", 
        "Space Opera",
        "Dark Fantasy", 
        "Western", 
        "Historique", 
        "Espionnage", 
        "Enquête",
        "Coopératif", 
        "Stratégie", 
        "Gestion", 
        "Deck-building", 
        "Placement d'ouvriers"
    ]
    
    # Insérer les genres dans la base de données
    genres_data = [(genre,) for genre in genres_list]
    
    # Utiliser INSERT IGNORE pour éviter les erreurs en cas de duplicata
    genres_sql = """
    INSERT IGNORE INTO genres (nom)
    VALUES (%s)
    """
    
    cursor.executemany(genres_sql, genres_data)
    conn.commit()
    print(f"✅ Genres créés avec succès!")
    
    # --- GÉNÉRATION DES ASSOCIATIONS JEUX-GENRES ---
    print("Association des jeux aux genres...")
    
    # Récupération des ids de genres
    cursor.execute("SELECT id FROM genres")
    genres_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES ASSOCIATIONS JEUX-GENRES ---
    print("Association des jeux aux genres...")
    
    # Récupération des ids de genres
    cursor.execute("SELECT id FROM genres")
    genres_ids = [row[0] for row in cursor.fetchall()]
    
    jeux_genres_data = []
    for jeu_id in jeux_ids:
        # Chaque jeu aura 1 à 3 genres aléatoires
        nombre_genres = min(random.randint(1, 3), len(genres_ids))
        genres_choisis = random.sample(genres_ids, nombre_genres)
        
        for genre_id in genres_choisis:
            jeux_genres_data.append((jeu_id, genre_id))
    
    # Éliminer les doublons
    jeux_genres_data = list(set(jeux_genres_data))
    
    jeux_genres_sql = """
    INSERT INTO jeux_genres (id_jeu, id_genre)
    VALUES (%s, %s)
    """
    
    cursor.executemany(jeux_genres_sql, jeux_genres_data)
    conn.commit()
    print(f"✅ {len(jeux_genres_data)} associations jeux-genres créées!")
    
    # --- GÉNÉRATION DES LIEUX ---
    print("Génération des lieux...")
    
    lieux_data = [
        ("Maison des Associations", "MDA", "123 Rue des Jeux, 45000 Orléans", "Salle principale de l'association", 47.902964, 1.909251),
        ("FSV", "FSV", "456 Avenue du Gaming, 45100 Orléans", "Fédération des Sports Virtuels", 47.913427, 1.873779),
        ("Discord", "DISC", "En ligne", "Serveur Discord de l'association", None, None),
        ("Bibliothèque Municipale", "BIB", "789 Boulevard des Livres, 45000 Orléans", "Salle de jeux au sous-sol", 47.899631, 1.900110),
        ("Café Ludique", "CAF", "10 Place du Plateau, 45000 Orléans", "Café avec jeux de société", 47.905783, 1.899988),
        ("Centre Culturel", "CC", "42 Avenue des Arts, 45000 Orléans", "Grande salle pour événements", 47.895631, 1.904233),
        ("Salle Polyvalente", "SP", "15 Rue des Rencontres, 45100 Orléans", "Espace modulable pour grands groupes", 47.920154, 1.887542),
        ("Roll20", "R20", "En ligne", "Plateforme de jeu de rôle virtuelle", None, None),
        ("Bar Le D20", "D20", "27 Rue du Hasard, 45000 Orléans", "Bar à thème jeux de société", 47.908743, 1.912564),
        ("Local Associatif", "LA", "8 Impasse des Joueurs, 45000 Orléans", "Petit local de l'association", 47.903126, 1.906873)
    ]
    
    lieux_sql = """
    INSERT INTO lieux (nom, short_nom, adresse, description, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(lieux_sql, lieux_data)
    conn.commit()
    print(f"✅ {len(lieux_data)} lieux créés avec succès!")
    
    # --- OBTENTION DES IDS DES LIEUX CRÉÉS ---
    cursor.execute("SELECT id FROM lieux")
    lieux_ids = [row[0] for row in cursor.fetchall()]
    
    # --- OBTENTION DES IDS DES UTILISATEURS CRÉÉS ---
    cursor.execute("SELECT id FROM utilisateurs")
    utilisateurs_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES ACTIVITÉS ---
    print("Génération des activités...")
    
    types_campagne = ["OUVERTE", "FERMEE"]
    
    activites_data = []
    
    # Séparer les jeux par type pour la génération contrôlée
    cursor.execute("SELECT id, nom, type_jeu FROM jeux")
    jeux_info_complete = cursor.fetchall()
    jeux_jdr = [jeu for jeu in jeux_info_complete if jeu[2] in ["JDR"]]
    jeux_societe = [jeu for jeu in jeux_info_complete if jeu[2] == "JEU_DE_SOCIETE"]
    
    # 1. Génération des activités JEU_DE_SOCIETE (quantité fixe)
    print(f"   Génération de {NOMBRE_JEUX_DE_SOCIETE} activités jeux de société...")
    for i in range(NOMBRE_JEUX_DE_SOCIETE):
        if jeux_societe:
            jeu_choisi = random.choice(jeux_societe)
            id_jeu = jeu_choisi[0]
            nom_jeu = jeu_choisi[1]
        else:
            # Fallback au cas où il n'y aurait pas assez de jeux de société
            id_jeu = random.choice(jeux_ids)
            nom_jeu = f"Jeu de société {i+1}"
            
        nom = f"Soirée {nom_jeu}"
        id_maitre_jeu = random.choice(utilisateurs_ids)
        type_activite = "JEU_DE_SOCIETE"
        type_campagne = None
        
        description_courte = f"Venez découvrir ou redécouvrir {nom_jeu} !"
        description = f"Une soirée conviviale autour de {nom_jeu}. Que vous soyez débutant ou expert, venez partager un moment ludique dans une ambiance détendue. Règles expliquées sur place !"
        nombre_max_joueurs = random.randint(4, 8)
        max_joueurs_session = nombre_max_joueurs
        verrouille = False  # Les jeux de société sont généralement ouverts
        
        # Récupérer l'image du jeu associé
        image = jeux_images_dict.get(id_jeu)
        texte_alt_image = f"Image pour l'activité {nom}" if image else None
        
        activites_data.append((
            nom, id_jeu, id_maitre_jeu, type_activite, type_campagne,
            description_courte, description, nombre_max_joueurs,
            max_joueurs_session, verrouille, image, texte_alt_image
        ))
    
    # 2. Génération des ÉVÉNEMENTS (quantité fixe)
    print(f"   Génération de {NOMBRE_EVENEMENTS} événements...")
    noms_evenements = [
        "Grande Convention Ludique", "Tournoi Inter-Associations", "Soirée Découverte",
        "Marathon JdR", "Festival du Jeu", "Concours de Création", "Soirée Déguisée",
        "Championnat Local", "Initiation Nouveaux Joueurs", "Salon du Jeu Indé"
    ]
    
    for i in range(NOMBRE_EVENEMENTS):
        nom = random.choice(noms_evenements) + f" #{i+1}"
        id_jeu = random.choice(jeux_ids)  # Un jeu peut être associé pour l'illustration
        id_maitre_jeu = random.choice(utilisateurs_ids)
        type_activite = "EVENEMENT"
        type_campagne = None
        
        description_courte = f"Un événement exceptionnel : {nom} !"
        description = f"{nom} est un événement spécial organisé par l'association. Venez nombreux pour participer à cette expérience unique qui rassemble toute la communauté ludique !"
        nombre_max_joueurs = random.randint(20, 50)  # Les événements accueillent plus de monde
        max_joueurs_session = random.randint(6, 12)  # Sessions en groupes plus petits
        verrouille = random.random() > 0.7  # Certains événements peuvent être sur inscription
        
        image = jeux_images_dict.get(id_jeu)
        texte_alt_image = f"Image pour l'événement {nom}" if image else None
        
        activites_data.append((
            nom, id_jeu, id_maitre_jeu, type_activite, type_campagne,
            description_courte, description, nombre_max_joueurs,
            max_joueurs_session, verrouille, image, texte_alt_image
        ))
    
    # 3. Génération des autres activités (CAMPAGNE et ONESHOT) pour atteindre le total
    activites_restantes = NOMBRE_ACTIVITES - NOMBRE_JEUX_DE_SOCIETE - NOMBRE_EVENEMENTS
    print(f"   Génération de {activites_restantes} campagnes et one-shots...")
    
    for i in range(activites_restantes):
        # Générer des noms plus variés pour les campagnes/oneshots
        noms_activites = [
            "L'aventure fantastique", "Les mystères de", "La quête de", "Les chroniques de",
            "L'épopée de", "Les légendes de", "Le secret de", "La guerre de",
            "L'empire de", "Les héros de", "La malédiction de", "Le trésor de"
        ]
        suffixes = [
            "la lumière perdue", "l'ombre éternelle", "la cité oubliée", "l'ancien royaume",
            "la forêt maudite", "l'étoile noire", "le dragon d'or", "la tour de cristal",
            "l'artefact légendaire", "la planète rouge", "l'alliance brisée", "l'élu"
        ]
        
        nom = f"{random.choice(noms_activites)} {random.choice(suffixes)}"
        
        # Privilégier les jeux JdR pour les campagnes/oneshots
        if jeux_jdr:
            jeu_choisi = random.choice(jeux_jdr)
            id_jeu = jeu_choisi[0]
        else:
            id_jeu = random.choice(jeux_ids)
            
        id_maitre_jeu = random.choice(utilisateurs_ids)
        type_activite = random.choice(["CAMPAGNE", "ONESHOT"])
        
        # Utilisation du paramètre pour déterminer le type de campagne
        if type_activite == "CAMPAGNE":
            if random.randint(1, 100) <= POURCENTAGE_CAMPAGNES_FERMEES:
                type_campagne = "FERMEE"
            else:
                type_campagne = "OUVERTE"
        else:
            type_campagne = None
            
        description_courte = f"Une aventure palpitante vous attend dans {nom.lower()} !"
        description = f"Description complète de {nom}. Une histoire épique avec de nombreux défis et mystères à résoudre. Les joueurs devront faire preuve de courage, d'intelligence et de coopération pour triompher des épreuves qui les attendent."
        nombre_max_joueurs = random.randint(6, 20) if type_campagne == "OUVERTE" else random.randint(3, 8)
        max_joueurs_session = min(nombre_max_joueurs, random.randint(2, 6))
        verrouille = random.random() > 0.8
        
        # Récupérer l'image du jeu associé pour l'utiliser comme image de l'activité
        image = jeux_images_dict.get(id_jeu)
        texte_alt_image = f"Image pour l'activité {nom}" if image else None
        
        activites_data.append((
            nom, id_jeu, id_maitre_jeu, type_activite, type_campagne,
            description_courte, description, nombre_max_joueurs,
            max_joueurs_session, verrouille, image, texte_alt_image
        ))
    
    activites_sql = """
    INSERT INTO activites (
        nom, id_jeu, id_maitre_jeu, type_activite, type_campagne,
        description_courte, description, nombre_max_joueurs,
        max_joueurs_session, verrouille, image, texte_alt_image
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(activites_sql, activites_data)
    conn.commit()
    print(f"✅ {len(activites_data)} activites créées avec succès!")
    
    # --- OBTENTION DES IDS DES PARTIES CRÉÉES ---
    cursor.execute("SELECT id FROM activites")
    activites_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES SESSIONS ---
    print("Génération des sessions...")

    sessions_data = []
    sessions_passees = 0
    sessions_futures = 0
    
    for activite_id in activites_ids:
        # Utilisation des paramètres pour le nombre de sessions par activité
        nombre_sessions = random.randint(SESSIONS_PAR_ACTIVITE_MIN, SESSIONS_PAR_ACTIVITE_MAX)
        
        for j in range(nombre_sessions):
            id_activite = activite_id
            id_lieu = random.choice(lieux_ids)
            
            # Déterminer si cette session sera dans le passé ou le futur
            if random.randint(1, 100) <= POURCENTAGE_SESSIONS_PASSEES:
                # Session dans le passé avec respect des jours autorisés
                date_session = generer_date_autorisee(is_passee=True)
                sessions_passees += 1
            else:
                # Session dans le futur avec respect des jours autorisés
                date_session = generer_date_autorisee(is_passee=False)
                sessions_futures += 1
            
            # Utilisation des paramètres pour les horaires avec gestion de l'heure 0h
            heure_debut_int = random.randint(HEURE_DEBUT_MIN, HEURE_DEBUT_MAX)
            
            # Gestion spéciale pour l'heure de fin avec support de minuit (0h)
            if HEURE_FIN_MIN == 0:
                # Cas spécial : on peut finir à minuit (0h) ou dans les premières heures
                # Créer une liste d'options valides
                options_fin = []
                
                # Ajouter les heures normales (au moins 2h après le début)
                heure_min_fin = max(heure_debut_int + 2, 1)
                if heure_min_fin <= 23:  # S'assurer qu'on ne dépasse pas 23h
                    options_fin.extend(range(heure_min_fin, 24))  # De heure_min_fin à 23h
                
                # Ajouter les premières heures du jour suivant (0h à HEURE_FIN_MAX)
                if HEURE_FIN_MAX >= 0:
                    options_fin.extend(range(0, HEURE_FIN_MAX + 1))
                
                # S'assurer qu'on a au moins une option
                if not options_fin:
                    options_fin = [0, 1, 2]  # Fallback par défaut
                
                heure_fin_int = random.choice(options_fin)
            else:
                # Logique normale : heure de fin dans la même journée
                heure_fin_int = random.randint(max(heure_debut_int + 2, HEURE_FIN_MIN), HEURE_FIN_MAX)
            
            heure_debut = f"{heure_debut_int:02d}:00:00"
            heure_fin = f"{heure_fin_int:02d}:00:00"
            
            # Récupérer le maître de jeu de l'activité
            cursor.execute("SELECT id_maitre_jeu FROM activites WHERE id = %s", (id_activite,))
            id_maitre_jeu = cursor.fetchone()[0]
            
            # Récupérer max_joueurs_session de l'activité
            cursor.execute("SELECT max_joueurs_session FROM activites WHERE id = %s", (id_activite,))
            max_joueurs_session = cursor.fetchone()[0]
            
            # Le nombre max de joueurs peut varier légèrement
            nombre_max_joueurs = max_joueurs_session if random.random() > 0.3 else random.randint(2, max_joueurs_session)
            
            # Générer un nom pour la session
            nom_session = f"Session {j+1} - {date_session}"
            
            sessions_data.append((
                nom_session, id_activite, id_lieu, date_session, heure_debut, heure_fin,
                id_maitre_jeu, nombre_max_joueurs
            ))

    sessions_sql = """
    INSERT INTO sessions (
        nom, id_activite, id_lieu, date_session, heure_debut, heure_fin,
        id_maitre_jeu, nombre_max_joueurs
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(sessions_sql, sessions_data)
    conn.commit()
    print(f"✅ {len(sessions_data)} sessions créées avec succès!")
    
    # --- OBTENTION DES IDS DES SESSIONS CRÉÉES ---
    cursor.execute("SELECT id FROM sessions")
    sessions_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES INSCRIPTIONS AUX SESSIONS ---
    print("Génération des inscriptions aux sessions...")
    
    # Dans la génération des inscriptions aux sessions
    joueurs_session_data = []
    for session_id in sessions_ids:
        # Récupérer le max_joueurs pour cette session
        cursor.execute("SELECT nombre_max_joueurs FROM sessions WHERE id = %s", (session_id,))
        max_joueurs = cursor.fetchone()[0]
        
        # Récupérer le maître de jeu de cette session pour ne pas l'inscrire comme joueur
        cursor.execute("SELECT id_maitre_jeu FROM sessions WHERE id = %s", (session_id,))
        id_maitre_jeu = cursor.fetchone()[0]
        
        # Vérifier les joueurs déjà inscrits à cette session
        cursor.execute("SELECT id_utilisateur FROM joueurs_session WHERE id_session = %s", (session_id,))
        joueurs_existants = [row[0] for row in cursor.fetchall()]
        
        # Utilisation du paramètre pour déterminer le nombre de joueurs
        nombre_joueurs_cible = int(max_joueurs * TAUX_INSCRIPTION_SESSIONS)
        # Ajouter un peu de variation aléatoire
        variation = random.randint(-1, 2)
        nombre_joueurs = max(0, min(max_joueurs, nombre_joueurs_cible + variation))
        
        joueurs_potentiels = [u for u in utilisateurs_ids if u != id_maitre_jeu and u not in joueurs_existants]
        
        if joueurs_potentiels and nombre_joueurs > 0:
            joueurs_selectionnes = random.sample(joueurs_potentiels, min(nombre_joueurs, len(joueurs_potentiels)))
            
            for joueur_id in joueurs_selectionnes:
                # Les inscriptions se font entre 1 et 30 jours avant la session
                date_inscription = datetime.now() - timedelta(days=random.randint(1, 30))
                joueurs_session_data.append((
                    session_id, joueur_id, date_inscription.strftime('%Y-%m-%d %H:%M:%S')
            ))
    
    joueurs_session_sql = """
    INSERT INTO joueurs_session (
        id_session, id_utilisateur, date_inscription
    ) VALUES (%s, %s, %s)
    """
    
    cursor.executemany(joueurs_session_sql, joueurs_session_data)
    conn.commit()
    print(f"✅ {len(joueurs_session_data)} inscriptions aux sessions créées!")
    
    # --- GÉNÉRATION DES MEMBRES DE PARTIES FERMÉES ---
    print("Génération des membres de activites fermées...")
    
    # Dans la activite génération des membres de activites fermées
    membres_activite_data = []
    for activite_id in activites_ids:
        # Vérifier le type d'activité et de campagne
        cursor.execute("SELECT type_activite, type_campagne, nombre_max_joueurs FROM activites WHERE id = %s", (activite_id,))
        result = cursor.fetchone()
        type_activite, type_campagne, max_joueurs = result
        
        if type_activite == "CAMPAGNE":
            # Récupérer le maître de jeu de cette activité
            cursor.execute("SELECT id_maitre_jeu FROM activites WHERE id = %s", (activite_id,))
            id_maitre_jeu = cursor.fetchone()[0]
            
            # Vérifier les membres déjà inscrits à cette activité
            cursor.execute("SELECT id_utilisateur FROM membres_activite WHERE id_activite = %s", (activite_id,))
            membres_existants = [row[0] for row in cursor.fetchall()]
            
            if type_campagne == "FERMEE":
                # Pour les campagnes fermées, on remplit plus
                nombre_membres = random.randint(max(1, int(max_joueurs * 0.6)), max_joueurs)
            else:  # OUVERTE
                # Pour les campagnes ouvertes, on utilise le taux de remplissage
                nombre_membres_cible = int(max_joueurs * TAUX_INSCRIPTION_SESSIONS)
                nombre_membres = random.randint(max(1, nombre_membres_cible - 2), min(max_joueurs, nombre_membres_cible + 2))
            
            membres_potentiels = [u for u in utilisateurs_ids if u != id_maitre_jeu and u not in membres_existants]
            
            if membres_potentiels:
                membres_selectionnes = random.sample(membres_potentiels, min(nombre_membres, len(membres_potentiels)))
                
                for membre_id in membres_selectionnes:
                    membres_activite_data.append((activite_id, membre_id))
    
    membres_activite_sql = """
    INSERT INTO membres_activite (id_activite, id_utilisateur)
    VALUES (%s, %s)
    """
    
    cursor.executemany(membres_activite_sql, membres_activite_data)
    conn.commit()
    print(f"✅ {len(membres_activite_data)} membres de activites fermées créés!")
    
    print("\n✅ GÉNÉRATION DES DONNÉES TERMINÉE AVEC SUCCÈS! ✅")
    
    # Afficher un résumé des données générées
    print("\n📊 Résumé des données générées:")
    print(f"   • {len(users_data)} utilisateurs")
    print(f"   • {len(jeux_data)} jeux")
    print(f"   • {len(genres_list)} genres")
    print(f"   • {len(lieux_data)} lieux")
    print(f"   • {len(activites_data)} activités")
    print(f"   • {len(sessions_data)} sessions")
    print(f"   • {len(joueurs_session_data)} inscriptions aux sessions")
    print(f"   • {len(membres_activite_data)} membres d'activités")
    
    # Statistiques des sessions par période
    cursor.execute("""
        SELECT DATE_FORMAT(date_session, '%Y-%m') as mois, COUNT(*) as nb_sessions 
        FROM sessions 
        GROUP BY DATE_FORMAT(date_session, '%Y-%m') 
        ORDER BY mois
    """)
    sessions_par_mois = cursor.fetchall()
    
    print("\n📅 Répartition des sessions par mois:")
    for mois, nb_sessions in sessions_par_mois:
        print(f"   • {mois}: {nb_sessions} sessions")
    
    # Statistiques des types d'activités
    cursor.execute("""
        SELECT type_activite, COUNT(*) as nombre 
        FROM activites 
        GROUP BY type_activite
    """)
    types_activites = cursor.fetchall()
    
    print("\n🎲 Répartition des activités par type:")
    for type_act, nombre in types_activites:
        print(f"   • {type_act}: {nombre}")
    
    # Statistiques des sessions par jour de la semaine
    cursor.execute("""
        SELECT 
            DAYNAME(date_session) as jour_semaine,
            WEEKDAY(date_session) as jour_num,
            COUNT(*) as nombre
        FROM sessions 
        GROUP BY DAYNAME(date_session), WEEKDAY(date_session)
        ORDER BY jour_num
    """)
    sessions_par_jour = cursor.fetchall()
    
    print("\n📅 Répartition des sessions par jour de la semaine:")
    jours_fr = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    for jour_en, jour_num, nombre in sessions_par_jour:
        jour_fr = jours_fr[jour_num]
        autorisation = "✅" if jour_num in JOURS_AUTORISES else "❌"
        print(f"   • {jour_fr}: {nombre} sessions {autorisation}")
    
    # Statistiques passé/futur
    cursor.execute("""
        SELECT 
            CASE 
                WHEN date_session < CURDATE() THEN 'Passées'
                ELSE 'Futures'
            END as periode,
            COUNT(*) as nb_sessions 
        FROM sessions 
        GROUP BY 
            CASE 
                WHEN date_session < CURDATE() THEN 'Passées'
                ELSE 'Futures'
            END
    """)
    sessions_par_periode = cursor.fetchall()
    
    print("\n⏰ Répartition temporelle des sessions:")
    for periode, nb_sessions in sessions_par_periode:
        print(f"   • {periode}: {nb_sessions} sessions")
    
    print("\n🎉 La base de données est maintenant remplie avec des données de test!")
    print(f"\n⚙️  Configuration utilisée:")
    print(f"   • Jours autorisés: {afficher_jours_autorises(JOURS_AUTORISES)}")
    print(f"   • Sessions passées: {SESSIONS_JOURS_PASSE_MIN}-{SESSIONS_JOURS_PASSE_MAX} jours ({POURCENTAGE_SESSIONS_PASSEES}%)")
    print(f"   • Sessions futures: {SESSIONS_JOURS_FUTUR_MIN}-{SESSIONS_JOURS_FUTUR_MAX} jours")
    print(f"   • Horaires: {HEURE_DEBUT_MIN}h-{HEURE_DEBUT_MAX}h → {HEURE_FIN_MIN}h-{HEURE_FIN_MAX}h")
    print(f"   • Taux de remplissage: {int(TAUX_INSCRIPTION_SESSIONS*100)}%")
except mysql.connector.Error as err:
    print(f"❌ Erreur: {err}")
finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("Connexion à la base de données fermée.")


Connexion à la base de données réussie!
✓ Table refresh_tokens vidée
✓ Table paiements_helloasso vidée
✓ Table notifications_helloasso vidée
✓ Table creneaux_utilisateur vidée
✓ Table horaires_lieu vidée
✓ Table joueurs_session vidée
✓ Table membres_activite vidée
✓ Table sessions vidée
✓ Table jeux_genres vidée
✓ Table evenements vidée
✓ Table activites vidée
✓ Table lieux vidée
✓ Table genres vidée
✓ Table jeux vidée
✓ Table periodes_association vidée
✓ Table utilisateurs vidée
✅ Toutes les tables ont été vidées avec succès!
Génération des utilisateurs...
✅ 20 utilisateurs créés avec succès!
Génération des jeux...
✅ 30 jeux créés avec succès!
Génération des genres...
✅ Genres créés avec succès!
Association des jeux aux genres...
Association des jeux aux genres...
✅ 59 associations jeux-genres créées!
Génération des lieux...
✅ 10 lieux créés avec succès!
Génération des activités...
   Génération de 30 activités jeux de société...
   Génération de 5 événements...
   Génération de 15 ca

# Script de génération de données de test pour la base de données LBDR

Ce notebook contient un script Python qui génère des données de test pour la base de données de l'application. Les données générées incluent :

- Des utilisateurs aléatoires
- Des jeux (JDR et jeux de société)
- Des genres associés aux jeux
- Des lieux pour les sessions
- Des activites (campagnes et one-shots)
- Des sessions liées aux activites
- Des inscriptions de joueurs aux sessions
- Des membres pour les campagnes fermées

Pour exécuter le script, assurez-vous que :
1. La base de données est accessible
2. Les tables ont été créées (le script ne crée pas les tables)
3. Le package `mysql-connector-python` est installé

Le script nettoie les tables existantes avant d'insérer de nouvelles données.